# Claude を用いたエンティティ抽出

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルで問題なく動作します。*

### コンテキスト
エンティティ抽出は、ニュース、電子メール、書籍などの自然に書かれたテキストから特定のデータを自動的に抽出できる NLP 手法です。
そのデータは、後にデータベースに保存し、検索やその他の処理に使用することができます。

従来のエンティティ抽出プログラムでは、通常、名前、住所、価格など、事前に定義済みのクラスに限定されるか、関心のあるエンティティの種類の例を多数提供する必要がありました。
エンティティ抽出に LLM を使用すると、ほとんどの場合、抽出する必要があるものを自然言語で指定するだけで済みます。これにより、データのラベル付けが不要になるため、時間を節約しながら、クエリを柔軟かつ正確に行うことができます。

さらに、LLM エンティティ抽出を使用すると、データセットを組み立てて、例えば [Amazon Comprehend のカスタムエンティティ](https://docs.aws.amazon.com/comprehend/latest/dg/custom-entity-recognition.html) の認識など、後にユースケースに合わせてカスタマイズされたソリューションの作成が可能です。

## セットアップ

このノートブックの残りの部分を実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrock に接続する必要があります。

セットアップの仕組みや ⚠️ **変更が必要か**については,  [Bedrock boto3 setup](../00_Intro/bedrock_boto3_setup.ipynb) ノートブックを参照してください。

このノートブックでは、いくつかの追加の依存関係も必要になります。:

- [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/)を使うと、Claude のプロンプトと出力で、XML タグからデータを簡単に抽出できます。

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57" \
    langchain==0.0.309 beautifulsoup4

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock


# ---- ⚠️ 必要に応じて AWS 設定に関する以下のコードのコメントを解除、編集してください  ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## langchain の設定

まず、LLM をインスタンス化します。ここでは、テキスト生成に Anthropic Claude v2 を使用しています。

注意 : Bedrockでは他のモデルも選択可能です。`model_id` を以下のように置き換えてモデルを変更することができます。

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

使用可能なモデル ID には以下が含まれます:

- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`
- `anthropic.claude-v2`

In [ ]:
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 200,
        "temperature": 0, # Using 0 to get reproducible results
        "stop_sequences": ["\n\nHuman:"]
    }
)

## エンティティ抽出
LLM が初期化されたので、エンティティの抽出を開始できます。

この演習では、質問や注文をメールで受け取るオンライン書店を想定します。
私たちの仕事は、メールから関連情報を抽出し、注文を処理することです。

まず、サンプルメールを見てみましょう。:

In [ ]:
from pathlib import Path

emails_dir = Path(".") / "emails"
with open(emails_dir / "00_treasure_island.txt") as f:
    book_question_email = f.read()

print(book_question_email)

### 基本のアプローチ

基本的なケースでは、モデルに直接結果を返すように依頼できます。
本の名前を抽出してみましょう。

In [ ]:
query = f"""

Human: Given the email inside triple-backticks, please read it and analyse the contents.
If a name of a book is mentioned, return it, otherwise return nothing.

Email: ```
{book_question_email}
```

Assistant:"""

In [ ]:
result = llm(query)
print(result.strip())

### モデル固有のプロンプト

基本的な方法でも問題ありませんが、最良の結果を得るには、使用する特定のモデルに合わせてプロンプトをカスタマイズすることをお勧めします。
今回の例では `anthropic.claude-v2`を使用しており、プロンプトガイドは[こちら](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)です。

Claude v2　のより最適化されたプロンプトは次のとおりです。

In [ ]:
prompt = """

Human: Given the email provided, please read it and analyse the contents.
If a name of a book is mentioned, return it.
If no name is mentioned, return empty string.
The email will be given between <email></email> XML tags.

<email>
{email}
</email>

Return the name of the book between <book></book> XML tags.

Assistant:"""

In [ ]:
query = prompt.format(email=book_question_email)
result = llm(query)
print(result.strip())

結果をより簡単に抽出するためには、補助関数を使うことができます。

In [ ]:
from bs4 import BeautifulSoup

def extract_by_tag(response: str, tag: str, extract_all=False) -> str | list[str] | None:
    soup = BeautifulSoup(response)
    results = soup.find_all(tag)
    if not results:
        return
        
    texts = [res.get_text() for res in results]
    if extract_all:
        return texts
    return texts[-1]

In [ ]:
extract_by_tag(result, "book")

We can check that our model doesn't return arbitrary results when no appropriate information is given (also know as 'hallucination'), by running our prompt on other emails.

他のメールにプロンプトを実行することで、適切な情報が与えられていない場合はモデルが任意の結果を返さないことを確認できます。（「ハルシネーション」とも呼ばれます）

In [ ]:
with open(emails_dir / "01_return.txt") as f:
    return_email = f.read()

print(return_email)

In [ ]:
query = prompt.format(email=return_email)
result = llm(query)
print(result.strip())

タグを使用すると、複数の情報を同時に抽出することもでき、抽出がより簡単になります。
次のプロンプトでは、本の名前だけでなく、質問、要求、顧客名も抽出します。

In [ ]:
prompt = """

Human: Given email provided , please read it and analyse the contents.

Please extract the following information from the email:
- Any questions the customer is asking, return it inside <questions></questions> XML tags.
- The customer full name, return it inside <name></name> XML tags.
- Any book names the customer mentions, return it inside <books></books> XML tags.

If a particular bit of information is not present, return an empty string.
Make sure that each question can be understoon by itself, incorporate context if requred.
Each returned question should be concise, remove extra information if possible.
The email will be given between <email></email> XML tags.

<email>
{email}
</email>

Return each question inside <question></question> XML tags.
Return the name of each book inside <book></book> XML tags.

Assistant:"""

In [ ]:
query = prompt.format(email=book_question_email)
result = llm(query)
print(result.strip())

In [ ]:
extract_by_tag(result, "question", extract_all=True)

In [ ]:
extract_by_tag(result, "name")

In [ ]:
extract_by_tag(result, "book", extract_all=True)

## まとめ

エンティティ抽出は、プレーンテキストの説明を使用して任意のデータを抽出することができる強力な手法です。

これは、明確な構造を持たない特定のデータを抽出する必要がある場合に特に便利です。その場合は、正規表現やその他の従来の抽出手法を実装するのは非常に難しい場合があります。

### 持ち帰り事項
- このノートブックを変更し、Amazon Bedrock を通して Amazon Titan や AI21 Labs Jurassic モデルなどの様々なモデルを試してみましょう。
- プロンプトを特定のユースケースに変更し、さまざまなモデルの出力を評価しましょう。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を得ましょう。推奨事項については、選択したモデルのプロンプトガイドを参照してください。例: Claude のプロンプトガイドは[こちら](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design).